In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.attacker import attackDatasetEditDist,changeRandomCharacters
import pandas as pd
import deepmatcher as dm
import numpy as np
from utils.deepmatcher_utils import wrapDm,getF1PrecisionRecall,getMeanConfidenceAndVariance
from tqdm import tqdm
from sklearn.utils import shuffle

## Define utilities

In [5]:
##given a perturbation model and number of perturbation calculate avg f1,precision and recall 
def getAvgF1PrecisionRecall(testpath,model,nperturbation,editdist=1):
    avgf1,avgrecall,avgprecision = 0,0,0
    for i in tqdm(range(nperturbation)):
        test_df = pd.read_csv(testpath)
        original_labels = test_df.label.values
        attacked_test = attackDatasetEditDist(test_df,['id','label'],editdist=editdist)
        attack_preds = wrapDm(attacked_test,model)
        f1,prec,recall = getF1PrecisionRecall(original_labels,attack_preds)
        avgf1 += f1
        avgprecision += prec
        avgrecall += recall
    return (avgf1/nperturbation,avgprecision/nperturbation,avgrecall/nperturbation)

In [6]:
def getAdversarialTrainingSet(train_df,dimension,editdist):
    shuffled_train = shuffle(train_df)
    samples_toalter = shuffled_train.iloc[:dimension].copy()
    unaltered = shuffled_train.iloc[dimension:].copy()
    attacked_train = attackDatasetEditDist(samples_toalter,['id','label'],editdist=1)
    newTrain = pd.concat([attacked_train,unaltered])
    return newTrain

## iTunes-Amazon

In [18]:
itunesamazon_mod = dm.MatchingModel(attr_summarizer='hybrid')
itunesamazon_mod.load_state('models/itunes_amazon_hybrid.pth')

In [23]:
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Structured/itunes-amazon/merged_test.csv',
                                                       itunesamazon_mod,10,editdist=1)
avgf1,avgprecision,avgrecall

100%|██████████| 10/10 [00:17<00:00,  1.72s/it]


(0.6282830787657788, 0.5985974796430859, 0.6629629629629629)

In [25]:
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Structured/itunes-amazon/merged_test.csv',
                                                       itunesamazon_mod,10,2)
avgf1,avgprecision,avgrecall

100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


(0.5812557450687164, 0.5592579985725148, 0.6074074074074074)

## Amazon-Google

In [28]:
amazongoogle_mod = dm.MatchingModel(attr_summarizer='hybrid')
amazongoogle_mod.load_state('models/amazongoogle_hybrid.pth')
amazongoogle_mod = amazongoogle_mod.to('cuda')

In [27]:
avgf1,avgrecall,avgprec = getAvgF1PrecisionRecall('datasets/Structured/Amazon-Google/merged_test.csv'
                                                  ,amazongoogle_mod,10)
avgf1,avgrecall,avgprec

100%|██████████| 10/10 [01:27<00:00,  8.70s/it]


(0.4883677183412362, 0.35589911623400244, 0.7782051282051282)

In [29]:
avgf1,avgrecall,avgprec = getAvgF1PrecisionRecall('datasets/Structured/Amazon-Google/merged_test.csv'
                                                  ,amazongoogle_mod,10,editdist=2)
avgf1,avgrecall,avgprec

100%|██████████| 10/10 [01:29<00:00,  8.95s/it]


(0.4364212892709801, 0.3319416337311589, 0.6371794871794872)

## Walmart-Amazon

In [61]:
walmartamazon_mod = dm.MatchingModel(attr_summarizer='hybrid')
walmartamazon_mod.load_state('models/walmartamazon_hybrid.pth')
walmartamazon_mod = walmartamazon_mod.to('cuda')

In [59]:
WALMART_DIR = 'datasets/Structured/Walmart-Amazon/'

In [31]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/Walmart-Amazon/merged_test.csv'
                                                  ,walmartamazon_mod,10)
avgf1,avgprec,avrecall

100%|██████████| 10/10 [01:58<00:00, 11.85s/it]


(0.10850399585294054, 0.6371794871794872, 0.06580310880829016)

In [32]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/Walmart-Amazon/merged_test.csv'
                                                  ,walmartamazon_mod,10,editdist=2)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [02:05<00:00, 12.50s/it]


(0.06535382037695106, 0.22951755907192792, 0.038341968911917094)

In [62]:
test_walmart = pd.read_csv(WALMART_DIR+'merged_test.csv')
getMeanConfidenceAndVariance(walmartamazon_mod,test_walmart)

(0.675714983248955, 0.007517719394529071)

## DBLP-ACM

In [46]:
dblpacm_mod = dm.MatchingModel(attr_summarizer='hybrid')
dblpacm_mod.load_state('models/dblp_acm_hybrid.pth')
dblpacm_mod = dblpacm_mod.to('cuda')

In [87]:
avgf1,avgrecall,avgprec = getAvgF1PrecisionRecall('datasets/Structured/DBLP-ACM/merged_test.csv'
                                                  ,dblpacm_mod,10)

In [88]:
avgf1,avgrecall,avgprec

(0.4466801600177579, 0.290990990990991, 0.9622991787108621)

In [89]:
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Structured/DBLP-ACM/merged_test.csv'
                                                  ,dblpacm_mod,10,editdist=2)

In [47]:
getMeanConfidenceAndVariance(dblpacm_mod,test_df)

(0.7988288628153603, 0.008252744318883462)

## Beer

In [24]:
beer_mod = dm.MatchingModel(attr_summarizer='hybrid')
beer_mod.load_state('models/beer_hybrid.pth')

In [105]:
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Structured/Beer/merged_test.csv'
                                                  ,beer_mod,10,editdist=1)
avgf1,avgrecall,avgprec

In [103]:
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Structured/Beer/merged_test.csv'
                                                  ,beer_mod,10,editdist=2)
avgf1,avgprecision,avgrecall

In [25]:
test_beer = pd.read_csv('datasets/Structured/Beer/merged_test.csv')
getMeanConfidenceAndVariance(beer_mod,test_beer)

(0.8160182391549204, 0.005981665059326744)

# Textual data

## Abt-Buy

In [1]:
abtbuy_model = dm.MatchingModel(attr_summarizer='hybrid')
abtbuy_model.load_state('models/abtbuy_hybrid.pth')

NameError: name 'dm' is not defined

In [94]:
%%capture
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Textual/Abt-Buy/merged_test.csv'
                                                  ,abtbuy_model,10)

In [95]:
avgf1,avgprecision,avgrecall

(0.5915975259056523, 0.6145631067961165, 0.5704149965608851)

In [96]:
%%capture
avgf1,avgprecision,avgrecall = getAvgF1PrecisionRecall('datasets/Textual/Abt-Buy/merged_test.csv'
                                                  ,abtbuy_model,10,editdist=2)

In [97]:
avgf1,avgprecision,avgrecall

(0.5167520023209786, 0.5393203883495146, 0.4962097058472218)

## Computers WDC

In [107]:
computer_model = dm.MatchingModel(attr_summarizer='rnn')
computer_model.load_state('models/computer_rnn.pth')

AttributeError: Can't get attribute '_default_unk_index' on <module 'torchtext.vocab' from '/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torchtext/vocab.py'>

# Retrain models

In [35]:
train_df = pd.read_csv('datasets/Structured/itunes-amazon/merged_train.csv')
train_df = shuffle(train_df)

In [39]:
newTrain.to_csv('datasets/Structured/itunes-amazon/attacked_train.csv',index=False)

In [42]:
train,validation,test = dm.data.process('datasets/Structured/itunes-amazon/',train='attacked_train.csv',
                                       validation='merged_valid.csv',test='merged_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "datasets/Structured/itunes-amazon/attacked_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/itunes-amazon/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/itunes-amazon/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [43]:
att_model = dm.MatchingModel(attr_summarizer='hybrid')
att_model.run_train(train,validation,epochs=20,pos_neg_ratio=5,
                    best_save_path='models/itunes_amazon_hybrid_att.pth')

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    8.3 | Load Time:    3.4 || F1:  43.24 | Prec:  27.79 | Rec:  97.44 || Ex/s:  54.93

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.6 || F1:  46.73 | Prec:  31.25 | Rec:  92.59 || Ex/s:  96.06

* Best F1: tensor(46.7290, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 2 || Run Time:    9.2 | Load Time:    3.9 || F1:  58.97 | Prec:  44.23 | Rec:  88.46 || Ex/s:  48.83

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.6 | Load Time:    0.6 || F1:  52.63 | Prec:  36.76 | Rec:  92.59 || Ex/s:  97.89

* Best F1: tensor(52.6316, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 19 || Run Time:    8.7 | Load Time:    3.5 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  52.54

===>  EVAL Epoch 19
Finished Epoch 19 || Run Time:    0.6 | Load Time:    0.6 || F1:  78.12 | Prec:  67.57 | Rec:  92.59 || Ex/s:  97.70

---------------------

===>  TRAIN Epoch 20


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 20 || Run Time:    8.9 | Load Time:    3.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  50.97

===>  EVAL Epoch 20
Finished Epoch 20 || Run Time:    0.6 | Load Time:    0.6 || F1:  78.12 | Prec:  67.57 | Rec:  92.59 || Ex/s:  97.79

---------------------

Loading best model...
Training done.


tensor(78.1250, device='cuda:0')

In [44]:
att_model.run_eval(test)

===>  EVAL Epoch 8
Finished Epoch 8 || Run Time:    0.6 | Load Time:    0.6 || F1:  83.64 | Prec:  82.14 | Rec:  85.19 || Ex/s:  95.99



tensor(83.6364, device='cuda:0')

In [45]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/itunes-amazon/merged_test.csv'
                                                  ,att_model,10)
avgf1,avgprec,avrecall

100%|██████████| 10/10 [00:15<00:00,  1.60s/it]


(0.7085311215597405, 0.6371794871794872, 0.7518518518518519)

In [46]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/itunes-amazon/merged_test.csv'
                                                  ,att_model,10,editdist=2)
avgf1,avgprec,avrecall

100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


(0.6838674248720723, 0.6371794871794872, 0.788888888888889)

### DBLP-ACM

In [25]:
train_df = pd.read_csv('datasets/Structured/DBLP-ACM/merged_train.csv')
adversarial_train = getAdversarialTrainingSet(train_df,int(len(train_df)/10),editdist=1)

In [29]:
adversarial_train.to_csv('datasets/Structured/DBLP-ACM/attacked_train.csv',index=False)

In [31]:
train,validation,test = dm.data.process('datasets/Structured/DBLP-ACM/',train='attacked_train.csv',
                                       validation='merged_validation.csv',test='merged_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "datasets/Structured/DBLP-ACM/attacked_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/DBLP-ACM/merged_validation.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/DBLP-ACM/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


In [32]:
att_model = dm.MatchingModel(attr_summarizer='hybrid')
att_model.run_train(train,validation,epochs=20,pos_neg_ratio=10,
                    best_save_path='models/dblpacm_hybrid_att.pth')

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:25


Finished Epoch 1 || Run Time:   58.2 | Load Time:   28.0 || F1:  64.18 | Prec:  47.39 | Rec:  99.40 || Ex/s:  86.09

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    5.2 | Load Time:    6.1 || F1:  92.60 | Prec:  86.21 | Rec: 100.00 || Ex/s: 217.33

* Best F1: tensor(92.5965, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:25


Finished Epoch 20 || Run Time:   57.9 | Load Time:   28.1 || F1:  99.70 | Prec:  99.40 | Rec: 100.00 || Ex/s:  86.27

===>  EVAL Epoch 20


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 20 || Run Time:    5.2 | Load Time:    6.1 || F1:  97.80 | Prec:  95.69 | Rec: 100.00 || Ex/s: 219.15

---------------------

Loading best model...
Training done.


tensor(97.9052, device='cuda:0')

In [33]:
att_model.run_eval(test)

===>  EVAL Epoch 9


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 9 || Run Time:    5.1 | Load Time:    6.0 || F1:  97.90 | Prec:  96.30 | Rec:  99.55 || Ex/s: 222.50



tensor(97.8959, device='cuda:0')

In [39]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/DBLP-ACM/merged_test.csv'
                                                  ,att_model,10)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [02:23<00:00, 14.32s/it]


NameError: name 'avgprec' is not defined

In [45]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/DBLP-ACM/merged_test.csv'
                                                  ,att_model,10,editdist=2)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [02:30<00:00, 15.04s/it]


(0.8567993710349715, 0.8380662415271702, 0.8765765765765765)

In [44]:
test_df = pd.read_csv('datasets/Structured/DBLP-ACM/merged_test.csv')
meanConfidence,confidenceVariance = getMeanConfidenceAndVariance(att_model,test_df)
meanConfidence,confidenceVariance

(0.7140764538763604, 0.01749880610028909)

### Walmart-Amazon

In [50]:
DATASET_DIR = 'datasets/Structured/Walmart-Amazon/'
train_df = pd.read_csv(DATASET_DIR+'merged_train.csv')
adversarial_train = getAdversarialTrainingSet(train_df,int(len(train_df)/10),editdist=1)
adversarial_train.to_csv(DATASET_DIR+'attacked_train.csv',index=False)

In [52]:
train,validation,test = dm.data.process(DATASET_DIR,train='attacked_train.csv',
                                       validation='merged_valid.csv',test='merged_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "datasets/Structured/Walmart-Amazon/attacked_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Walmart-Amazon/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Walmart-Amazon/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [54]:
att_model = dm.MatchingModel(attr_summarizer='hybrid')
att_model.run_train(train,validation,epochs=20,pos_neg_ratio=10,
                    best_save_path='models/walmartamazon_hybrid_att.pth')

* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:10


Finished Epoch 1 || Run Time:   55.8 | Load Time:   16.0 || F1:  20.54 | Prec:  11.52 | Rec:  94.79 || Ex/s:  85.46

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    5.2 | Load Time:    3.4 || F1:  23.23 | Prec:  13.21 | Rec:  95.85 || Ex/s: 237.53

* Best F1: tensor(23.2266, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:10


Finished Epoch 20 || Run Time:   55.7 | Load Time:   15.8 || F1:  98.55 | Prec:  97.13 | Rec: 100.00 || Ex/s:  85.84

===>  EVAL Epoch 20


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 20 || Run Time:    5.1 | Load Time:    3.4 || F1:  60.28 | Prec:  54.89 | Rec:  66.84 || Ex/s: 239.92

---------------------

Loading best model...
Training done.


tensor(60.6897, device='cuda:0')

In [55]:
att_model.run_eval(test)

===>  EVAL Epoch 13


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 13 || Run Time:    5.1 | Load Time:    3.4 || F1:  64.89 | Prec:  60.91 | Rec:  69.43 || Ex/s: 242.37



tensor(64.8910, device='cuda:0')

In [63]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/Walmart-Amazon/merged_test.csv'
                                                  ,att_model,10)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [01:59<00:00, 11.94s/it]


(0.19217939506029855, 0.37355021968771074, 0.1295336787564767)

In [57]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall('datasets/Structured/Walmart-Amazon/merged_test.csv'
                                                  ,att_model,10,editdist=2)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [02:05<00:00, 12.56s/it]


(0.13452835825365744, 0.29218230747456314, 0.08756476683937822)

In [58]:
test_walmart = pd.read_csv(DATASET_DIR+'merged_test.csv')
getMeanConfidenceAndVariance(att_model,test_walmart)

(0.6878982216861552, 0.008079620766869848)

### Amazon-Google

In [66]:
GOOGLE_DIR = 'datasets/Structured/Amazon-Google/'
train_df = pd.read_csv(GOOGLE_DIR+'merged_train.csv')
adversarial_train = getAdversarialTrainingSet(train_df,int(len(train_df)/7),editdist=1)
adversarial_train.to_csv(GOOGLE_DIR+'attacked_train.csv',index=False)

In [68]:
train,validation,test = dm.data.process(GOOGLE_DIR,train='attacked_train.csv',
                                       validation='merged_valid.csv',test='merged_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "datasets/Structured/Amazon-Google/attacked_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Amazon-Google/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Amazon-Google/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [69]:
att_model = dm.MatchingModel(attr_summarizer='hybrid')
att_model.run_train(train,validation,epochs=22,pos_neg_ratio=10,
                    best_save_path='models/amazongoogle_hybrid_att.pth')

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:52


Finished Epoch 1 || Run Time:   40.2 | Load Time:   12.4 || F1:  23.15 | Prec:  13.23 | Rec:  92.42 || Ex/s: 130.61

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    3.7 | Load Time:    2.5 || F1:  27.22 | Prec:  15.80 | Rec:  98.29 || Ex/s: 371.94

* Best F1: tensor(27.2189, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:52


Finished Epoch 22 || Run Time:   40.3 | Load Time:   12.5 || F1:  98.52 | Prec:  97.08 | Rec: 100.00 || Ex/s: 130.16

===>  EVAL Epoch 22


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 22 || Run Time:    3.5 | Load Time:    2.4 || F1:  65.08 | Prec:  66.08 | Rec:  64.10 || Ex/s: 388.00

---------------------

Loading best model...
Training done.


tensor(67.3597, device='cuda:0')

In [70]:
att_model.run_eval(test)

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.5 | Load Time:    2.4 || F1:  64.67 | Prec:  64.81 | Rec:  64.53 || Ex/s: 393.40



tensor(64.6681, device='cuda:0')

In [71]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall(GOOGLE_DIR+'merged_test.csv'
                                                  ,att_model,10)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [01:19<00:00,  7.90s/it]


(0.42456651270331347, 0.5195466305483931, 0.3594017094017094)

In [72]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall(GOOGLE_DIR+'merged_test.csv'
                                                  ,att_model,10,2)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [01:23<00:00,  8.35s/it]


(0.36569272561297456, 0.4561403873257742, 0.3055555555555555)

In [73]:
test_df = pd.read_csv(GOOGLE_DIR+'merged_test.csv')
getMeanConfidenceAndVariance(att_model,test_df)

(0.679582653038376, 0.007873293631689001)

### Beer

In [17]:
BEER_DIR = 'datasets/Structured/Beer/'
train_df = pd.read_csv(BEER_DIR+'merged_train.csv')
adversarial_train = getAdversarialTrainingSet(train_df,int(len(train_df)/10),editdist=1)
adversarial_train.to_csv(BEER_DIR+'attacked_train.csv',index=False)

In [18]:
train,validation,test = dm.data.process(BEER_DIR,train='attacked_train.csv',
                                       validation='merged_valid.csv',test='merged_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "datasets/Structured/Beer/attacked_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Beer/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "datasets/Structured/Beer/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [19]:
att_model = dm.MatchingModel(attr_summarizer='hybrid')
att_model.run_train(train,validation,epochs=22,pos_neg_ratio=4,
                    best_save_path='models/beer_hybrid_att.pth')

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    2.1 | Load Time:    0.6 || F1:  21.43 | Prec:  15.00 | Rec:  37.50 || Ex/s:  97.53

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.2 || F1:  37.74 | Prec:  25.64 | Rec:  71.43 || Ex/s: 192.91

* Best F1: tensor(37.7359, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 2 || Run Time:    2.1 | Load Time:    0.6 || F1:  59.38 | Prec:  43.18 | Rec:  95.00 || Ex/s:  97.02

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.2 || F1:  46.81 | Prec:  33.33 | Rec:  78.57 || Ex/s: 193.51

* Best F1: tensor(46.8085, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 21 || Run Time:    2.2 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  92.42

===>  EVAL Epoch 21
Finished Epoch 21 || Run Time:    0.3 | Load Time:    0.2 || F1:  70.59 | Prec:  60.00 | Rec:  85.71 || Ex/s: 187.23

---------------------

===>  TRAIN Epoch 22


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 22 || Run Time:    2.2 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  92.57

===>  EVAL Epoch 22
Finished Epoch 22 || Run Time:    0.3 | Load Time:    0.2 || F1:  70.59 | Prec:  60.00 | Rec:  85.71 || Ex/s: 188.04

---------------------

Loading best model...
Training done.


tensor(70.5882, device='cuda:0')

In [20]:
att_model.run_eval(test)

===>  EVAL Epoch 9
Finished Epoch 9 || Run Time:    0.3 | Load Time:    0.2 || F1:  72.73 | Prec:  63.16 | Rec:  85.71 || Ex/s: 187.29



tensor(72.7273, device='cuda:0')

In [21]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall(BEER_DIR+'merged_test.csv'
                                                  ,att_model,10)
avgf1,avgprecision,avrecall

100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


(0.6456010956010956, 0.516166080296515, 0.8642857142857142)

In [26]:
avgf1,avgprecision,avrecall = getAvgF1PrecisionRecall(BEER_DIR+'merged_test.csv'
                                                  ,att_model,20,editdist=2)
avgf1,avgprecision,avrecall

100%|██████████| 20/20 [00:12<00:00,  1.63it/s]


(0.5453974354237625, 0.39783692509261764, 0.8714285714285716)

In [23]:
test_df = pd.read_csv(BEER_DIR+'merged_test.csv')
getMeanConfidenceAndVariance(att_model,test_df)

(0.8009747127582739, 0.008585936948849798)